In [127]:
#reloads source files to get most recent edits
import importlib
importlib.reload(getImageArrays)

<module 'getImageArrays' from '../Functions/getImageArrays.py'>

In [128]:
import sys
import numpy as np
import tensorflow as tf
sys.path.append("../Functions/")
# sys.path.remove("../Functions/")
from getImageArrays import getExamples
from importlib import reload

In [129]:
image_path = "../Data/Logistic_Regression_Data/"
num_images = 100
examples_per_batch = 16

In [130]:
train_x, train_y, test_x, test_y = getExamples(side_length=100, image_path=image_path , test_ratio=.2, max_num_images=num_images)
train_x, train_y, test_x, test_y  = train_x.T, train_y.T, test_x.T, test_y.T
assert((len(train_x) + len(test_x)) == num_images), "Not sufficeint number of images" 

Error in creating examples notcow197.jpg


In [131]:
def createBatches(x, examples_per_batch):
    final = []
    for i in range(0, len(x), examples_per_batch):
        start = i
        end = start + examples_per_batch
        #creates exclusively
        if end >= len(x):
            return np.array(final)
        final += [x[start:end]]
    return np.array(final)
#         final += x[]

In [132]:
train_x, train_y = createBatches(train_x, examples_per_batch), createBatches(train_y, examples_per_batch)
test_x, test_y  = createBatches(test_x, examples_per_batch), createBatches(test_y, examples_per_batch)

In [133]:
print("train_x", train_x.shape)
print("train_y", train_y.shape)
print("test_x", test_x.shape)
print("test_y", test_y.shape)

train_x (4, 16, 30000)
train_y (4, 16, 1)
test_x (1, 16, 30000)
test_y (1, 16, 1)
